In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from urllib.request import urlopen
import ssl

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE


In [2]:
lists=[]
description=[]

for k in range(1,301):
    html_text=requests.get('https://www.flipkart.com/search?q=mobiles&sid=tyy%2C4io&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_1_5_na_na_na&otracker1=AS_QueryStore_OrganicAutoSuggest_1_5_na_na_na&as-pos=1&as-type=RECENT&suggestionId=mobiles%7CMobiles&requestId=a6ac8ea7-69a2-49fc-a761-09d905c75e5f&as-searchtext=mobil&page={}'.format(k)).text
    soup=BeautifulSoup(html_text,'lxml')
    #print(soup)
        
    mobiles=soup.find_all('div',class_="_1AtVbE col-12-12")
    #print(mobiles)
    for mobile in mobiles:
            review=mobile.find('div', class_="_3LWZlK")
            mobi=mobile.find('div', class_="_4rR01T")
            price=mobile.find('div', class_="_30jeq3 _1_WHN1")
            details=mobile.find_all('li', class_="rgWa7D")
            de=[]
            for d in details:

                
                de.append(d.text)
            if mobi and review is not None:
                description.append(de)
                #print(d)
            
                lists.append([mobi.text,review.text,price.text,de])
    if k%50==0:
        print(' iteration done',format(k))
                

 iteration done 50
 iteration done 100
 iteration done 150
 iteration done 200
 iteration done 250
 iteration done 300


In [3]:
print(len(lists))


964


In [4]:

df=pd.DataFrame(data=lists,columns=['mobile_name','rating','price','det'])
print(df.shape)
df.head()

(964, 4)


,mobile_name,rating,price,det
0,"realme C20 (Cool Blue, 32 GB)",4.4,"₹7,499",[2 GB RAM | 32 GB ROM | Expandable Upto 256 GB...
1,"realme C20 (Cool Grey, 32 GB)",4.4,"₹7,499",[2 GB RAM | 32 GB ROM | Expandable Upto 256 GB...
2,"realme C21 (Cross Blue, 64 GB)",4.4,"₹9,799",[4 GB RAM | 64 GB ROM | Expandable Upto 256 GB...
3,"realme C21 (Cross Black, 64 GB)",4.4,"₹9,799",[4 GB RAM | 64 GB ROM | Expandable Upto 256 GB...
4,"realme C21 (Cross Blue, 32 GB)",4.4,"₹8,799",[3 GB RAM | 32 GB ROM | Expandable Upto 256 GB...


In [5]:
df.det[960]

['32 MB RAM | 32 MB ROM | Expandable Upto 32 GB',
 '6.1 cm (2.4 inch) Quarter QVGA Display',
 '0.3MP Rear Camera',
 '1800 mAh Battery',
 '1 Year Brand Warranty']

In [7]:
df['display']=None
df['memory']=None
df['warrenty']=None
df['camera']=None
df['other_details']=None

for i in range(964):
    df['camera'][i]=df.det[i][2]
    df['display'][i]=df.det[i][1]
    df['warrenty'][i]=df.det[i][-1]
    df['memory'][i]=df.det[i][0]
    df['other_details'][i]=df.det[i][3:-1]
    

In [8]:
df.isnull().sum()

mobile_name      0
rating           0
price            0
det              0
display          0
memory           0
warrenty         0
camera           0
other_details    0
dtype: int64

In [9]:
df.drop(['det'],axis=1,inplace=True)
    

In [10]:
df.head()

,mobile_name,rating,price,display,memory,warrenty,camera,other_details
0,"realme C20 (Cool Blue, 32 GB)",4.4,"₹7,499",16.51 cm (6.5 inch) HD+ Display,2 GB RAM | 32 GB ROM | Expandable Upto 256 GB,1 Year Warranty for Mobile and 6 Months for Ac...,8MP Rear Camera | 5MP Front Camera,"[5000 mAh Battery, MediaTek Helio G35 Processor]"
1,"realme C20 (Cool Grey, 32 GB)",4.4,"₹7,499",16.51 cm (6.5 inch) HD+ Display,2 GB RAM | 32 GB ROM | Expandable Upto 256 GB,1 Year Warranty for Mobile and 6 Months for Ac...,8MP Rear Camera | 5MP Front Camera,"[5000 mAh Battery, MediaTek Helio G35 Processor]"
2,"realme C21 (Cross Blue, 64 GB)",4.4,"₹9,799",16.51 cm (6.5 inch) HD+ Display,4 GB RAM | 64 GB ROM | Expandable Upto 256 GB,1 Year Warranty for Mobile and 6 Months for Ac...,13MP + 2MP + 2MP | 5MP Front Camera,"[5000 mAh Battery, MediaTek Helio G35 Processor]"
3,"realme C21 (Cross Black, 64 GB)",4.4,"₹9,799",16.51 cm (6.5 inch) HD+ Display,4 GB RAM | 64 GB ROM | Expandable Upto 256 GB,1 Year Warranty for Mobile and 6 Months for Ac...,13MP + 2MP + 2MP | 5MP Front Camera,"[5000 mAh Battery, MediaTek Helio G35 Processor]"
4,"realme C21 (Cross Blue, 32 GB)",4.4,"₹8,799",16.51 cm (6.5 inch) HD+ Display,3 GB RAM | 32 GB ROM | Expandable Upto 256 GB,1 Year Warranty for Mobile and 6 Months for Ac...,13MP + 2MP + 2MP | 5MP Front Camera,"[5000 mAh Battery, MediaTek Helio G35 Processor]"


In [11]:
df.to_csv('mobile.csv')